In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import NuSVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import random

# Load the dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip"
!wget -q $url -O har.zip
!unzip -qq har.zip
features = pd.read_csv("UCI HAR Dataset/features.txt", delim_whitespace=True, header=None)
X = pd.read_csv("UCI HAR Dataset/train/X_train.txt", delim_whitespace=True, header=None)
X.columns = features[1]
y = pd.read_csv("UCI HAR Dataset/train/y_train.txt", header=None).values.ravel()

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Parameter search space
kernels = ['rbf', 'poly', 'sigmoid']
nus = np.linspace(0.1, 0.9, 9)
epsilons = np.linspace(0.001, 0.2, 10)

results = []
convergence_data = []

for sample in range(10):
    # Split into train/test (70/30)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=sample)

    best_acc = 0
    best_params = None
    acc_progress = []

    for iteration in range(100):
        kernel = random.choice(kernels)
        nu = round(random.choice(nus), 2)
        epsilon = round(random.choice(epsilons), 3)

        try:
            model = NuSVC(kernel=kernel, nu=nu)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
        except:
            acc = 0  # skip invalid combos

        acc_progress.append(acc)

        if acc > best_acc:
            best_acc = acc
            best_params = (kernel, nu, epsilon)

    results.append({
        "Sample": f"S{sample+1}",
        "Accuracy": round(best_acc * 100, 2),
        "Kernel": best_params[0],
        "Nu": best_params[1],
        "Epsilon": best_params[2],
        "Acc_Progress": acc_progress
    })

# Save Table 1
df_results = pd.DataFrame(results)
df_results_display = df_results[['Sample', 'Accuracy', 'Kernel', 'Nu', 'Epsilon']]
print("\n=== Table 1: Comparative Performance ===")
print(df_results_display.to_string(index=False))

# Plot convergence for best sample
best_index = df_results['Accuracy'].idxmax()
plt.figure(figsize=(10,5))
plt.plot(range(1, 101), results[best_index]['Acc_Progress'], label="Accuracy")
plt.title(f"Convergence of Best SVM (Sample: {results[best_index]['Sample']})")
plt.xlabel("Iteration")
plt.ylabel("Accuracy")
plt.grid(True)
plt.legend()
plt.show()


<ipython-input-3-6ad8578daf42>:14: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  features = pd.read_csv("UCI HAR Dataset/features.txt", delim_whitespace=True, header=None)
<ipython-input-3-6ad8578daf42>:15: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X = pd.read_csv("UCI HAR Dataset/train/X_train.txt", delim_whitespace=True, header=None)
